# Chapter 4. Searching and Reading Local Files
### Crawling and searching directories (p. 114)
Download [dir](https://github.com/PacktPublishing/Python-Automation-Cookbook-Second-Edition/tree/master/Chapter04/documents/dir) using DownGit.   [os.path doc](https://docs.python.org/3/library/os.path.html) [OO high-level os.pathlib doc](https://docs.python.org/3/librarpathlib.html)

In [12]:
import os, re
for root, dirs,files in os.walk('dir'):
    print(f'     root={root}')
    for file in files:
        if file.endswith('.pdf'):
            print('PDF===>',end=' ')
        if re.search(r'[13579]', file):
            print('ODD===>',end=' ')
        full_path = os.path.join(root,file)
        print(file, full_path)

     root=dir
file2.txt dir/file2.txt
ODD===> file1.txt dir/file1.txt
PDF===> file6.pdf dir/file6.pdf
     root=dir/subdir
ODD===> file3.txt dir/subdir/file3.txt
file4.txt dir/subdir/file4.txt
PDF===> ODD===> file5.pdf dir/subdir/file5.pdf


### Reading text files (p. 117)
downloaded [zen_of_python.txt](https://github.com/PacktPublishing/Python-Automation-Cookbook-Second-Edition/blob/master/Chapter04/documents/zen_of_python.txt)

In [20]:
with open('zen_of_python.txt') as file:
    for line in file:
        if 'should' in line.lower():
            print(line) 
        if 'dutch' in line.lower():
            print(line)
            break

Errors should never pass silently.

There should be one-- and preferably only one --obvious way to do it.

Although that way may not be obvious at first unless you're Dutch.



### Dealing with encodings (p. 120)
downloaded documents dir. [Encoding aliases](https://docs.python.org/3/library/codecs.html#standard-encodings)

In [ ]:
with open('documents/example_utf8.txt') as file:
    print(file.read())
with open('documents/example_iso.txt', encoding='iso-8859-1') as file:
    print(file.read())

In [ ]:
with open('documents/example_utf8.txt') as file:
    content = file.read()
with open('documents/junk_output_iso.txt', 'w', encoding='iso-8859-1') as file:
    file.write(content)
with open('documents/junk_output_iso.txt', encoding='iso-8859-1') as file:
    print(file.read())

In [32]:
from bs4 import UnicodeDammit
with open('documents/example_iso.txt', 'rb') as file:
    content = file.read()
suggestion = UnicodeDammit(content)
suggestion.original_encoding
suggestion.unicode_markup

'20£'

### Reading CSV files
[csv docs](https://docs.python.org/3/library/csv.html)

In [ ]:
import csv
with open('documents/top_films.csv') as file:
    data = csv.reader('file')
    for row in file:
        print(row)

In [66]:
#with open('documents/top_films.csv') as file:
file =open('documents/top_films.csv')
#  treats first line as headers with KEYs
data = csv.DictReader(file)
structured_data = [row for row in data]
structured_data[0]
structured_data[0].keys()
structured_data[0]['Rank']
file.close()

['excel', 'excel-tab', 'unix']

In [ ]:
csv.list_dialects()
with open('documents/top_films.csv', newline='') as file:
    dialect = csv.Sniffer().sniff(file.read())
    
with open('documents/top_films.csv', newline='') as file:
    reader = csv.reader(file, dialect)
    for row in reader:
        print(row)